In [ ]:
import numpy as np 
import os
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.resnet import *
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.utils import plot_model

import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import os, os.path
import functools
import cv2
from PIL import Image

from google.colab.patches import cv2_imshow
from google.colab import drive

In [ ]:
def conv_act(inputs,out_filters,activation='relu'):
  return Conv2D(filters = out_filters , activation=activation, kernel_size=3, strides=1, padding='same')(inputs)

In [ ]:
def decoder(inputs, mid_filters=512, out_filters=256, activation='relu', block_name='decoder'):
  with K.name_scope(block_name):
    conv = conv_act(inputs,mid_filters,activation)
    conv_tr = Conv2DTranspose(filters = out_filters, activation=activation, kernel_size=4, strides=2, padding='same')(conv)
    return conv_tr

In [ ]:
num_classes = 2
input_shape = (256,256,3)
resnet101 = ResNet101(include_top=False, weights='imagenet', classes=num_classes, input_shape= input_shape)
resnet101.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
for layer in resnet101.layers:
   layer.trainable = False

In [ ]:
pool = MaxPooling2D(2,strides=2)(resnet101.get_output_at(0))
# output = decoder(pool,block_name='decoder_center')
dec_center = decoder(pool,block_name='decoder_center')
cat1 = Concatenate()([dec_center,resnet101.get_layer('conv5_block3_out').output])
# output = decoder(cat1,block_name='decoder1')
dec5 = decoder(cat1,block_name='decoder5')
cat2 = Concatenate()([dec5,resnet101.get_layer('conv4_block23_out').output])
dec4 = decoder(cat2,block_name='decoder4')
cat3 = Concatenate()([dec4,resnet101.get_layer('conv3_block3_out').output])
dec3 = decoder(cat3,256,64,block_name='decoder3')
cat2 = Concatenate() ([dec3,resnet101.get_layer('conv2_block2_out').output])
dec2 = decoder(cat2,128,128,block_name='decoder2')
dec1 = decoder(dec2,128,32,block_name='decoder1')
dec0 = conv_act(dec1,32)
output = Conv2D(1,1,activation='sigmoid',padding='same')(dec0)

In [ ]:
model = Model(input= [resnet101.get_input_at(0)], output=output)
optimizer = Adam(lr = 1e-3,decay = 1e-6,amsgrad=False)
#optimizer = SGD(lr = 1e-1,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(optimizer = optimizer , loss = 'binary_crossentropy', metrics = ['accuracy'])
# Visually inspect new model to confirm it the correct architecture
model.summary()

In [ ]:
plot_model(model)

## Try to use resnet101_v2



In [ ]:
#resnet101_v2 = ResNet101V2(include_top=False, weights='imagenet',classes=2)

In [ ]:
#resnet101_v2.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#resnet101_v2.summary()

## Resnet101

In [ ]:
def get_layer_by_name(layer_list,name):
  return [layer for layer in layer1_list if layer.name == name] [0]

In [ ]:
sel_layer = get_layer_by_name(layer1_list,'conv2_block1_3_conv')
sel_layer.get_config()m

In [ ]:
def build_block_layer(layer_list):
    def block_layer(x):
        for layer in layer_list:
            x = layer(x)
        return x
    return block_layer

In [ ]:
def build_block_layer(layer_list,**kwargs):
  return Lambda(lambda x: functools.reduce(lambda t, layer: layer(t), layer_list, x), **kwargs)

In [ ]:
from torchvision import models
model = models.resnet101()
print(model)

In [ ]:
model.summary()

In [ ]:
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/My Drive/EPFL_2019/ML_project_2/"
raw_img_path = path + "training/images/"
raw_masks_path = path + "training/groundtruth/"
rsz_img_path = path + "training/rsz_images/"
rsz_masks_path = path + "training/rsz_groundtruth/"
n = len([name for name in os.listdir(raw_img_path)])
print("Number of training images: " + str(n))

In [ ]:
# Resize images
not_rsz_imgs = []
for i in range(1,n+1):
    dim = (256, 256) #(w,h)
    if i < 10:
        filename = "satImage_00" + str(i) + ".png"
    elif i < 100:
        filename = "satImage_0" + str(i) + ".png"  
    else: 
        filename = "satImage_" + str(i) + ".png"
    filepath = raw_img_path + filename
    image = cv2.imread(filepath,1)
    try:
        resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    except:
        print("Cannot resize: " + str(filename))
        not_rsz_imgs.append(filename)
    (thresh, im_bw) = cv2.threshold(resized, 128, 255, cv2.THRESH_BINARY)
    cv2.imwrite(rsz_img_path + filename, im_bw)
print(not_rsz_imgs)

In [ ]:
# Resize masks
not_rsz_masks = []
for i in range(1,n+1):
    dim = (256, 256) #(w,h)
    if i < 10:
        filename = "satImage_00" + str(i) + ".png"
    elif i<100:
        filename = "satImage_0" + str(i) + ".png"  
    else: 
        filename = "satImage_" + str(i) + ".png"
    filepath = raw_masks_path + filename
    image = cv2.imread(filepath, 0)
    try:
        resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    except:
        print("Cannot resize: " + str(filename))
        not_rsz_imgs.append(filename)
    (thresh, im_bw) = cv2.threshold(resized, 128, 255, cv2.THRESH_BINARY)
    cv2.imwrite(rsz_masks_path + filename, im_bw)
print(not_rsz_imgs)

In [ ]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

def adjustData(img,gt):
    #make sure img value is between 0 and 1, and that mask is either 1 or 0
    if(np.max(img) > 1):
        img = img / 255
        gt = gt /255
        gt[gt > 0.5] = 1
        gt[gt <= 0.5] = 0
    return (img,gt)

def trainGenerator(batch_size,path,image_folder,gt_folder,augmentation_variables,save_to_dir = None,
                    target_size = (256,256),seed = 1):

    #create generators generating coresponding images using same seed, that will yield coresponding images
    image_generator = ImageDataGenerator(validation_split=0.2,**augmentation_variables)
    gt_generator = ImageDataGenerator(validation_split=0.2,**augmentation_variables)
    
#     print(path)
#     print([image_folder])
#     print(gt_folder)
    image_generator = image_generator.flow_from_directory(
        path,
        classes = [image_folder],
        class_mode = None,
        color_mode = "rgb",
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        seed = seed)
    
    gt_generator = gt_generator.flow_from_directory(
        path,
        classes = [gt_folder],
        class_mode = None,
        color_mode = "grayscale",
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        seed = seed)
    
    #zip both generators into a shared generator
    train_generator = zip(image_generator, gt_generator)
    #adjust image values to be between 0 and 1, and gt images to be 0 or 1
    for (image,gt) in train_generator:
        image,gt = adjustData(image,gt)
        yield (image,gt)

In [ ]:
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)

imgs_train = []
imgs_mask_train = []
for i in range(1, n+1):
    if i < 10:
          filename = "satImage_00" + str(i) + ".png"
    elif i<100:
          filename = "satImage_0" + str(i) + ".png"  
    else: 
          filename = "satImage_" + str(i) + ".png"
    filepath = rsz_img_path + filename
    img = Image.open(filepath)
    arr = np.array(img)
    imgs_train.append(arr)
    filepath = rsz_masks_path + filename
    img = Image.open(filepath)
    arr = np.array(img)
    arr = np.expand_dims(arr, -1)
    imgs_mask_train.append(arr)

imgs_train = np.array(imgs_train)
imgs_mask_train = np.array(imgs_mask_train)

imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255  # scale masks to [0, 1]

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
# tensorboard = TensorBoard(log_dir='tensorboard/', write_graph=True, write_images=True)

In [ ]:
generator_variables = dict(rotation_range=90,
                           width_shift_range=0.4,
                           height_shift_range=0.4,
                           zoom_range=0.5,
                           horizontal_flip = True,
                           vertical_flip = True,
                           fill_mode='reflect')
batch_size = 128
steps_per_epoch = 10
epochs = 100
train_generator = trainGenerator(batch_size,path+"/training/","rsz_images","rsz_groundtruth",generator_variables)

In [ ]:
print('*'*30)
print('Fitting model...')
print('*'*30)
# history = model.fit(imgs_train, imgs_mask_train, batch_size=8, epochs=30, verbose=2, shuffle=True,
history = model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs=epochs, verbose=2, shuffle=True,callbacks=[model_checkpoint])

In [ ]:
file = h5py.File('Dataset_test.h5', 'r')
imgs_test = file.get('images')
#imgs_mask_test = file.get('masks')
imgs_test = np.array(imgs_test)
#imgs_mask_test = np.array(imgs_mask_test)
imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

print('*'*30)
print('Loading saved weights...')
print('*'*30)
model.load_weights('weights.h5')

print('*'*30)
print('Predicting masks on test data...')
print('*'*30)
imgs_mask_test = model.predict(imgs_test, verbose=1)

In [ ]:
print('*' * 30)
print('Saving predicted masks to files...')
print('*' * 30)
pred_dir = 'Preds2'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for i, image in enumerate(imgs_mask_test):
    image = (image * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(pred_dir, str(i + 1) + '_pred.png'), image)

In [ ]:
plt.figure(figsize=(60, 30))
plt.plot(history.history['loss'], linewidth=8, color='r')                   #visualising training and validation loss curves
plt.plot(history.history['val_loss'], linewidth=8, color='b')
plt.title('Model train vs Validation Loss', fontsize=100, fontweight="bold")
plt.ylabel('Loss', fontsize=80)
plt.xlabel('Epoch', fontsize=80)
plt.legend(['Train', 'Validation'], loc='upper right', fontsize=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.show()